In [1]:
!pip install torch lightning numpy kaggle wandb torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.7/777.7 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.7 MB/s eta 0:00:00


In [2]:
from google.colab import files

# Carica il file kaggle.json
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"alexxxyy47","key":"29be390563ab164c9d7671a10a031097"}'}

In [3]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [4]:
!kaggle datasets download -d ealaxi/paysim1
!unzip paysim1.zip
!rm paysim1.zip

 97% 172M/178M [00:01<00:00, 204MB/s]
100% 178M/178M [00:01<00:00, 157MB/s]
Archive:  paysim1.zip
  inflating: PS_20174392719_1491204439457_log.csv  


In [72]:
import pandas as pd, sys, plotly.graph_objects as go, plotly.express as px, numpy as np, torch, random as rnd, torch.nn as nn, lightning as L, wandb as wndb, pdb
from torch.utils.data import Dataset, DataLoader
from sklearn.utils import shuffle
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
from matplotlib import pyplot as plt
import networkx as nx
from torch_geometric.nn import GCNConv
import torch_geometric.transforms as T
from torch import optim
from torchmetrics.classification import BinaryF1Score
from torch_geometric.nn import MessagePassing
from typing import Optional
from torch.nn import Parameter
from torch_geometric.typing import (
    Adj,
    OptPairTensor,
    OptTensor,
    SparseTensor,
    torch_sparse,
)
from torch_geometric.utils import add_remaining_self_loops
from torch_geometric.utils import add_self_loops as add_self_loops_fn
from torch_geometric.utils import (
    is_torch_sparse_tensor,
    scatter,
    spmm,
    to_edge_index,
)
from torch_geometric.utils.num_nodes import maybe_num_nodes
from torch_geometric.utils.sparse import set_sparse_value
from torch_geometric.nn.dense.linear import Linear
from torch_geometric.nn.inits import zeros
from torchmetrics.functional.classification import binary_accuracy, binary_precision, binary_recall


from sklearn.preprocessing import StandardScaler
from torch.nn.functional import binary_cross_entropy_with_logits

In [73]:
# PARAMETERS

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
ACCELERATOR =  "gpu" if torch.cuda.is_available() else "cpu"
SEED = 42
PRECISION = torch.float

rnd.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

DEPTH = 3
DEPTH_LINEAR = 2
LR = 1e-3
WD = 1e-5
H_SIZE = 32

"aaf831dabc88d936d4e6b439b798bb4cb42814ea"

#wndb.login()



'aaf831dabc88d936d4e6b439b798bb4cb42814ea'

In [3]:
# UTILS FUNCTIONS

def load_dataframe( dataset_file : str):
    return pd.read_csv(dataset_file)


def find_null_or_empty_records( dataframe: pd.DataFrame):
    n = len(dataframe)
    for index, row in dataframe.iterrows():
        print_progress_bar(index/n)
        # Controlla se ci sono valori nulli o vuoti nel record
        if row.isnull().any() or any(map(lambda x: x == '', row)):
            # Stampa il record
            print(f"Record con valori nulli o vuoti:\n{row}\n")

def print_progress_bar(percentuale, lunghezza_barra=20):
    blocchi_compilati = int(lunghezza_barra * percentuale)
    barra = "[" + "=" * (blocchi_compilati - 1) + ">" + " " * (lunghezza_barra - blocchi_compilati) + "]"
    sys.stdout.write(f"\r{barra} {percentuale * 100:.2f}% completo")
    sys.stdout.flush()


def compute_kind_inconsistence(dataframe):
    return {"inconsistent orig balance": len(dataframe.query('abs(oldbalanceOrg - newbalanceOrig) != amount'))/len(dataframe),
            "inconsistent dest balance": len(dataframe.query('abs(oldbalanceDest - newbalanceDest) != amount'))/len(dataframe),
            "zero cash transaction": len(dataframe.query('amount == 0 '))/len(dataframe),
            "self-transaction": len(dataframe.query('nameOrig == nameDest'))/len(dataframe)
            }

def plot_histogram(to_plot):


    # Converti il dizionario in un array di valori
    values = list(to_plot.values())

    # Crea un istogramma
    fig = go.Figure(data=[go.Bar(x=list(to_plot.keys()), y=values)])

    # Mostra l'istogramma
    fig.show()



def plot_categories(dataframe):
    # Calcola la frequenza di ogni categoria nella colonna 'type'
    counts = dataframe['type'].value_counts().reset_index()

    # Rinomina le colonne
    counts.columns = ['type', 'count']

    counts['count'] = counts['count'] / counts['count'].sum()

    # Crea l'istogramma con Plotly Express
    fig = px.bar(counts, x='type', y='count', title='Istogramma delle categorie nella colonna "type"')

    # Mostra il plot
    fig.show()


def build_names_conversion_index(df):
  all_names = pd.concat([df['nameOrig'], df['nameDest']])


  # Applica factorize per ottenere identificatori univoci
  unique_names, name_indices = pd.factorize(all_names)


  # Costruisci il dizionario di mapping
  name_to_id = dict(zip(name_indices, unique_names))
  #breakpoint()
  return name_to_id


def random_split_ids(id_list, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, seed=None):



    # Set the random seed for reproducibility
    rnd.seed(seed)

    # Shuffle the ID list
    shuffled_ids = rnd.sample(id_list, len(id_list))

    # Calculate the split indices
    train_size = int(train_ratio * len(shuffled_ids))
    val_size = int(val_ratio * len(shuffled_ids))

    # Split the shuffled IDs
    train_set = shuffled_ids[:train_size]
    val_set = shuffled_ids[train_size:train_size + val_size]
    test_set = shuffled_ids[train_size + val_size:]

    return train_set, val_set, test_set


def create_weights_tensor(original_tensor, n, m):
    # Crea un nuovo tensore con la stessa forma del tensore

    new_tensor = torch.full_like(original_tensor, m)

    # Imposta i valori a n dove il tensore originale ha valore 1
    new_tensor[original_tensor == 1] = n

    return new_tensor


In [21]:
class FraudDetectionDataset(Dataset):

    def __init__(self,dataset_file : str):
        dataframe = load_dataframe(dataset_file)
        dataframe["isFraud"] = dataframe["isFraud"].astype(int)
        dataframe["isFlaggedFraud"] = dataframe["isFlaggedFraud"].astype(int)

        self.raw_data = dataframe
        self.transaction_types = {
            "CASH_IN": 0,
            "CASH_OUT": 1,
            "DEBIT": 2,
            "PAYMENT": 3,
            "TRANSFER": 4
        }

    def analize_data(self,find_empty_records = True):
        print("----HEAD----")
        print(self.raw_data.head())
        print("----INFO----")
        print(self.raw_data.info())
        print("----DESCRIBE----")
        print(self.raw_data.describe())
        if find_empty_records:
          find_null_or_empty_records(self.raw_data)

    def extract_inconsistent_transactions(self):
        condiction = "abs(oldbalanceOrg - newbalanceOrig) != amount | abs(oldbalanceDest - newbalanceDest) != amount | amount == 0 | nameOrig == nameDest"

        return self.raw_data.query(condiction)


    def remove_inconsistent(self):
        breakpoint()
        self.raw_data = self.raw_data.query("amount != 0") #abs(oldbalanceOrg - newbalanceOrig) == abs(oldbalanceDest - newbalanceDest) &

        pos = self.raw_data.query("isFraud == 1")
        neg = self.raw_data.query("isFraud == 0")
        neg = neg.iloc[:len(pos)]
        self.raw_data = pd.concat([pos, neg], axis=0)


    def convert_type(self):
      scaler = StandardScaler()
      self.raw_data['type'] = self.raw_data['type'].apply(lambda x: self.transaction_types[x])
      self.raw_data['step'] = self.raw_data['step'].apply(lambda x: x%24)
      #breakpoint()
      mean = self.raw_data['oldbalanceOrg'].mean()
      self.raw_data['oldbalanceOrg'] = self.raw_data['oldbalanceOrg'].apply(lambda x: x/mean)
      mean = self.raw_data['oldbalanceDest'].mean()
      self.raw_data['oldbalanceDest'] = self.raw_data['oldbalanceDest'].apply(lambda x: x/mean)
      # self.raw_data['oldbalanceOrg'] = scaler.fit_transform(self.raw_data['oldbalanceOrg'])
      # self.raw_data['oldbalanceDest'] = scaler.fit_transform(self.raw_data['oldbalanceDest'].values.reshape(-1, 1))




    def build_graph(self, val_prc, test_prc):
      self.ids = [i for i in range(len(self.raw_data))]
      self.train_ids, self.val_ids, self.test_ids = random_split_ids(self.ids, val_ratio=val_prc, test_ratio=test_prc)

      edge_index = []
      #breakpoint()
      node_names = build_names_conversion_index(self.raw_data)
      id = 0
      x = []
      self.raw_data['nameOrig'] = self.raw_data['nameOrig'].apply(lambda x: node_names[x])
      self.raw_data['nameDest'] = self.raw_data['nameDest'].apply(lambda x: node_names[x])
      ordered_items = sorted(list(node_names.items()), key=lambda x: x[1])
      for node in ordered_items:
        x.append([1 if node[0].startswith('M') else 0])

      edge_attr = torch.tensor(self.raw_data[['step', 'type', 'amount', 'oldbalanceOrg', 'oldbalanceDest']].values, dtype=PRECISION)
      y = torch.tensor(self.raw_data['isFraud'].to_numpy(), dtype=PRECISION)
      edge_index = torch.tensor(self.raw_data[['nameOrig', 'nameDest']].values, dtype=torch.int).t()
      self.raw_data = None
      x = torch.tensor(x, dtype=PRECISION)

      #breakpoint()
      data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
      data.y = y
      self.split = T.RandomNodeSplit(num_val=val_prc, num_test=test_prc)
      self.data = [self.split(data)]


    def __getitem__(self, index):
      return self.data[index]

    def __len__(self):
      return 1


    def collate_train(self, data):

      #breakpoint()
      return data[0], self.train_ids

    def collate_val(self, data):

      #breakpoint()
      return data[0], self.val_ids



    def get_dataloader(self, validation=False):
      return DataLoader(self, batch_size=1, shuffle=False, collate_fn = self.collate_val if validation else self.collate_train)


    def to_device(self):
      #breakpoint()
      self.data[0].x.to(DEVICE)
      self.data[0].y.to(DEVICE)
      self.data[0].edge_attr.to(DEVICE)
      self.data[0].edge_index.to(DEVICE)
      self.data[0].train_mask.to(DEVICE)
      self.data[0].val_mask.to(DEVICE)
      self.data[0].test_mask.to(DEVICE)









In [22]:
dataset = FraudDetectionDataset("PS_20174392719_1491204439457_log.csv")
#inconsistent_data = dataset.extract_inconsistent_transactions()

In [7]:
dataset.raw_data.head


<bound method NDFrame.head of Empty DataFrame
Columns: [step, type, amount, nameOrig, oldbalanceOrg, newbalanceOrig, nameDest, oldbalanceDest, newbalanceDest, isFraud, isFlaggedFraud, step, type, amount, nameOrig, oldbalanceOrg, newbalanceOrig, nameDest, oldbalanceDest, newbalanceDest, isFraud, isFlaggedFraud]
Index: []

[0 rows x 22 columns]>

In [ ]:
d = compute_kind_inconsistence(inconsistent_data)
plot_histogram(d)
names_distinct = set(dataset.raw_data["nameOrig"]).union(set(dataset.raw_data["nameDest"]))

In [23]:
print(dataset.raw_data.columns)
dataset.convert_type()
dataset.remove_inconsistent()
dataset.build_graph(0.1,0.2)




Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud'],
      dtype='object')
> <ipython-input-21-de1d119b745b>(35)remove_inconsistent()
     33     def remove_inconsistent(self):
     34         breakpoint()
---> 35         self.raw_data = self.raw_data.query("amount != 0") #abs(oldbalanceOrg - newbalanceOrig) == abs(oldbalanceDest - newbalanceDest) &
     36 
     37         pos = self.raw_data.query("isFraud == 1")

ipdb> c



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.10/bdb.py", line 347, in set_continue
    sys.settrace(None)



In [ ]:
def convert_to_networkx(graph):
	g = to_networkx(graph, node_attrs=['x'])

	return g

def plot_graph(g):
	plt.figure(figsize=(9,7))
	nx.draw_spring(g, node_size=30, arrows=False)
	plt.show()



g = convert_to_networkx(dataset.data[0])
plot_graph(g)
del g


In [74]:
class FraudDetectionModule(L.LightningModule):

  def __init__(self,nodes_size,edges_size, hidden_layer_size_gnn,hidden_layer_size_edges , depth_gnn, depth_edges, lr=1e-3, wd=5e-4):
    super().__init__()
    self.layers = []

    self.layers.append(GCNConv(nodes_size, hidden_layer_size_gnn).to(DEVICE))
    for _ in range(depth_gnn - 2):
      self.layers.append(GCNConv(hidden_layer_size_gnn, hidden_layer_size_gnn).to(DEVICE))
    print("conv layers")
    for l in self.layers:
      print(list(l.parameters())[0])
    print("linear layers")
    if hidden_layer_size_edges < 2:
      self.edges_nn = nn.Linear(edges_size,1)
      print(self.edges_nn.weight)
    else:
      edges_layers = [nn.Linear(edges_size,hidden_layer_size_edges),nn.ELU()]
      for _ in range(depth_edges-2):
        edges_layers.append(nn.Linear(hidden_layer_size_edges, hidden_layer_size_edges))
        edges_layers.append(nn.ELU())

      edges_layers.append(nn.Linear(hidden_layer_size_edges,1))
      self.edges_nn = nn.Sequential(*edges_layers)
      print(self.edges_nn[0].weight)
      print(self.edges_nn[2].weight)


    self.classifier = nn.Linear(2*hidden_layer_size_gnn+1, 1)
    self.relu = nn.ELU()
    self.sigmoid = nn.Sigmoid()
    self.lr = lr
    self.wd = wd
    self.validation_step_outputs = []
    self.f1 = BinaryF1Score()



    self.loss = nn.BCEWithLogitsLoss()


  def forward(self,data):
    edge_index = data[0].edge_index
    edge_attr =  data[0].edge_attr #nn.functional.normalize(data[0].edge_attr) #data[0].edge_attr
    x = data[0].x

    train_edges = data[1]

    if torch.isinf(edge_index).any().item():
      print("edge_index_inf")
      exit(1)
    if (torch.isnan(edge_index ).any().item()):
      print("edge_index")
      exit(1)
    if torch.isinf(edge_attr).any().item():
      print("edge_attr_inf 1")
      exit(1)
    if (torch.isnan(edge_attr ).any().item()):
      print("edge_attr 1")
      exit(1)
    if torch.isinf(x).any().item():
      print("x_inf 1")
      exit(1)
    if (torch.isnan(x ).any().item()):
      print("x 1")
      exit(1)

    #breakpoint()
    #print(edge_attr)
    # print("edges weight ",self.edges_nn[0].weight)
    # print("conv weight ", list(self.layers[0].parameters())[0])

    edge_attr = self.edges_nn(edge_attr)
    if torch.isinf(edge_attr).any().item():
      print("edge_attr_inf 2")
      exit(1)
    if (torch.isnan(edge_attr ).any().item()):

      print("edge_attr 2")
      print(edge_attr)
      exit(1)

    for cnn in self.layers:
      x = self.relu(cnn(x, edge_index, edge_attr))
    # if (torch.isnan(x ).any().item()):
    #   print("x 2")
    #   exit(1)
    train_features = edge_attr[train_edges]
    train_edge_index = edge_index[:,train_edges]
    del edge_attr
    del train_edges
    del edge_index
    #breakpoint()
    #add batch  with edge nodes + features of edges
    from_nodes = torch.nan_to_num(x.squeeze()[train_edge_index[0,:].squeeze()])
    dest_nodes = torch.nan_to_num(x.squeeze()[train_edge_index[1,:].squeeze()])
    del train_edge_index
    to_classify = torch.cat((from_nodes,dest_nodes,train_features), dim=1)
    if torch.isinf(to_classify).any().item():
      print("to_classify_inf 2")
      exit(1)
    if (torch.isnan(to_classify ).any().item()):
      print("to_classify")
      exit(1)
    del from_nodes
    del dest_nodes
    return self.sigmoid(self.classifier(to_classify).squeeze())



  def training_step(self, batch, batch_idx):
    #breakpoint()

    #print("training")
    z = self.forward(batch)
    y = batch[0].y[batch[1]]
    w = create_weights_tensor(y,1,0.7)
    #breakpoint()
    loss = binary_cross_entropy_with_logits(z,y,weight=w)



    print("train_loss = ",float(loss))
    # Logging to TensorBoard (if installed) by default
    self.log("train_loss", loss, prog_bar=True)
    opt = self.optimizers()

    # compute loss


    opt.zero_grad()
    self.manual_backward(loss)
    #print("Gradiente del primo parametro:", self.edges_nn[0].weight.grad)

    # clip gradients
    self.clip_gradients(opt, gradient_clip_val=0.5, gradient_clip_algorithm="norm")

    opt.step()


  def validation_step(self, batch, batch_idx):
    with torch.no_grad():
      #breakpoint()

      #print("validation")

      z = self.forward(batch)
      print("z = ",list(z))
      #breakpoint()
      val_loss = self.loss(z,batch[0].y[batch[1]])
      acc = binary_accuracy(z, batch[0].y[batch[1]])
      prec = binary_precision(z, batch[0].y[batch[1]])
      rec = binary_recall(z, batch[0].y[batch[1]])
      print("acc = ", float(acc))
      print("prec = ", float(prec))
      print("rec = ", float(rec))
      print("val_loss = ", float(val_loss))
      f1 = self.f1(z,batch[0].y[batch[1]])
      print("f1-score = ", float(f1))
      #wndb.log({"val_loss": val_loss,"f1-score":f1})
      self.log_dict({"val_loss": val_loss,"f1-score":f1}, prog_bar=True)


  def configure_optimizers(self):
    optimizer = optim.Adam(self.parameters(), lr=self.lr, weight_decay=self.wd)
    return optimizer




In [81]:


# wndb.init(
#     project="datamining-hw4",

#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": LR ,
#     "weight decay": WD ,
#     "layers": DEPTH ,
#     "hidden layers size": H_SIZE
#     })
print(DEVICE)
#dataset.to_device()
train_loader = dataset.get_dataloader()
val_loader = dataset.get_dataloader(validation=True)
#breakpoint()


#model.to(DEVICE)

trainer = L.Trainer(deterministic=True,max_epochs=100,accelerator=ACCELERATOR)



INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


cuda


In [86]:
model = FraudDetectionModule(dataset.data[0].num_node_features,dataset.data[0].edge_attr.shape[1] ,H_SIZE,H_SIZE,DEPTH,DEPTH_LINEAR,LR,WD)
model.automatic_optimization = False

conv layers
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0', requires_grad=True)
linear layers
Parameter containing:
tensor([[ 0.2109, -0.0490, -0.0120,  0.3014, -0.2545],
        [-0.3139,  0.2902,  0.4285,  0.3382,  0.2765],
        [-0.2376,  0.2696, -0.2310,  0.2003,  0.4059],
        [-0.1337, -0.3301, -0.3177, -0.1494,  0.0331],
        [ 0.4094,  0.0816, -0.2288, -0.0443, -0.1265],
        [ 0.1106,  0.1803, -0.2605,  0.1449,  0.2167],
        [-0.0275, -0.3083,  0.2853, -0.4260,  0.2196],
        [ 0.0706, -0.1230, -0.1703,  0.0989, -0.2580],
        [-0.0191,  0.3334, -0.0278, -0.2981,  0.3486],
        [ 0.2786,  0.3689, -0.1379, -0.4379, -0.1

In [87]:
# for name, param in model.named_parameters():
    # print(f"Parameter {name} device:", param.device)



trainer.fit(model, train_loader, val_loader)
#wndb.finish() 1131626


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name       | Type              | Params
-------------------------------------------------
0 | edges_nn   | Sequential        | 225   
1 | classifier | Linear            | 66    
2 | relu       | ELU               | 0     
3 | sigmoid    | Sigmoid           | 0     
4 | f1         | BinaryF1Score     | 0     
5 | loss       | BCEWithLogitsLoss | 0     
-------------------------------------------------
291       Trainable params
0         Non-trainable params
291       Total params
0.001     Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name       | Type              | Params
-------------------------------------------------
0 | edges_nn   | Sequential        | 225   
1 | classifier | Linear            | 66    
2 | relu       | ELU               | 0     
3 | sigmoid    | Sigmoid           | 0     
4 | f1

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

z =  [tensor(1., device='cuda:0'), tensor(1., device='cuda:0'), tensor(1., device='cuda:0'), tensor(1., device='cuda:0'), tensor(1., device='cuda:0'), tensor(1., device='cuda:0'), tensor(1., device='cuda:0'), tensor(1., device='cuda:0'), tensor(1., device='cuda:0'), tensor(1., device='cuda:0'), tensor(1., device='cuda:0'), tensor(1., device='cuda:0'), tensor(1., device='cuda:0'), tensor(1.0000, device='cuda:0'), tensor(0.9999, device='cuda:0'), tensor(1., device='cuda:0'), tensor(1., device='cuda:0'), tensor(1., device='cuda:0'), tensor(1., device='cuda:0'), tensor(1., device='cuda:0'), tensor(1., device='cuda:0'), tensor(1., device='cuda:0'), tensor(1., device='cuda:0'), tensor(1., device='cuda:0'), tensor(1., device='cuda:0'), tensor(1., device='cuda:0'), tensor(1., device='cuda:0'), tensor(1., device='cuda:0'), tensor(1., device='cuda:0'), tensor(1., device='cuda:0'), tensor(1., device='cuda:0'), tensor(1., device='cuda:0'), tensor(1., device='cuda:0'), tensor(1., device='cuda:0'), 

Training: |          | 0/? [00:00<?, ?it/s]

train_loss =  0.613936722278595


...

edge_attr 2
tensor([[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]], device='cuda:0')
to_classify
z =  [tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(na

Validation: |          | 0/? [00:00<?, ?it/s]

edge_attr 2
tensor([[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]], device='cuda:0')
to_classify
z =  [tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(nan, device='cuda:0'), tensor(na

Validation: |          | 0/? [00:00<?, ?it/s]

edge_attr 2
tensor([[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]], device='cuda:0')
to_classify
z =  

In [ ]:
dataset.analize_data(find_empty_records=False)
print("inconsistent fraud")
print(len(inconsistent_data.query('isFraud == 1'))/len(inconsistent_data))
print("inconsistent not fraud")
print(len(inconsistent_data.query('isFraud == 0'))/len(inconsistent_data))
print("given > gotten")
print(len(dataset.raw_data.query("abs(oldbalanceOrg - newbalanceOrig) > abs(oldbalanceDest - newbalanceDest)"))/len(inconsistent_data))
print("given < gotten")
print(len(dataset.raw_data.query("abs(oldbalanceOrg - newbalanceOrig) < abs(oldbalanceDest - newbalanceDest)"))/len(inconsistent_data))
print("distinct origin name")
print(dataset.raw_data["nameOrig"].nunique())
print("distinct names")
print(len(names_distinct))
print("isFlaggedFraud and isFraud")
print(len(dataset.raw_data.query("isFlaggedFraud & isFraud"))/len(dataset.raw_data.query("isFraud")))
plot_categories(dataset.raw_data)
